In [31]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [17]:
%pip install tensorflow-model-analysis

Note: you may need to restart the kernel to use updated packages.


In [19]:
import tensorflow_model_analysis as tfma

INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.Sequence[~T]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:absl:tensorflow_ranking is not available: No module named 'tensorflow_ranking'
INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'
INFO:absl:tensorflow_decision_forests is not available: No module named 'tensorflow_decision_forests'
INFO:absl:struct2tensor is not available: No module named 'struct2tensor'
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using An

In [1]:
from google.cloud import aiplatform
from google.cloud import aiplatform_v1

In [2]:
pipeline_name = "fraud-detect-pipeline"
project_id = "ihr-vertex-pipelines"
region = "europe-west4"
aiplatform.init(project=project_id, location=region)

In [3]:
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(region)
metadata_client = aiplatform_v1.MetadataServiceClient(
  client_options={
      "api_endpoint": API_ENDPOINT
  }
)

In [4]:
LIVE_FILTER = "create_time > \"2022-02-27T14:30:00+10:00\" AND state = LIVE"

artifact_request = aiplatform_v1.ListArtifactsRequest(
    parent="projects/{0}/locations/{1}/metadataStores/default".format(project_id, region),
	filter=LIVE_FILTER)

In [5]:
artifacts = metadata_client.list_artifacts(artifact_request)

In [7]:
artifacts

ListArtifactsPager<artifacts {
  name: "projects/237148598933/locations/europe-west4/metadataStores/default/artifacts/1614169409458270259"
  display_name: "evaluation"
  uri: "gs://ihr-vertex-pipelines/payoneer/pipeline/237148598933/fraud-detect-pipeline-20220227143821/Evaluator_6702046738829541376/evaluation"
  etag: "1645969239320"
  create_time {
    seconds: 1645969106
    nanos: 177000000
  }
  update_time {
    seconds: 1645969239
    nanos: 320000000
  }
  state: LIVE
  schema_title: "tfx.ModelEvaluation"
  schema_version: "0.0.1"
  metadata {
    fields {
      key: "tfx_version"
      value {
        string_value: "1.6.1"
      }
    }
  }
}
artifacts {
  name: "projects/237148598933/locations/europe-west4/metadataStores/default/artifacts/7493935592862427244"
  display_name: "blessing"
  uri: "gs://ihr-vertex-pipelines/payoneer/pipeline/237148598933/fraud-detect-pipeline-20220227143821/Evaluator_6702046738829541376/blessing"
  etag: "1645969239304"
  create_time {
    seconds:

In [20]:
evaluation = [x for x in artifacts if x.display_name == 'evaluation'][0]

In [27]:
evaluation

name: "projects/237148598933/locations/europe-west4/metadataStores/default/artifacts/1614169409458270259"
display_name: "evaluation"
uri: "gs://ihr-vertex-pipelines/payoneer/pipeline/237148598933/fraud-detect-pipeline-20220227143821/Evaluator_6702046738829541376/evaluation"
etag: "1645969239320"
create_time {
  seconds: 1645969106
  nanos: 177000000
}
update_time {
  seconds: 1645969239
  nanos: 320000000
}
state: LIVE
schema_title: "tfx.ModelEvaluation"
schema_version: "0.0.1"
metadata {
  fields {
    key: "tfx_version"
    value {
      string_value: "1.6.1"
    }
  }
}

In [28]:
eval_result = tfma.load_eval_result(evaluation.uri)

In [29]:
eval_result

EvalResult(slicing_metrics=[((), {'': {'': {'binary_accuracy': {'doubleValue': 0.9989346265792847}, 'loss': {'doubleValue': 0.004936288110911846}}}})], plots=[((), None)], attributions=[((), None)], config=model_specs {
  label_key: "Class"
}
slicing_specs {
}
metrics_specs {
  model_names: ""
  per_slice_thresholds {
    key: "binary_accuracy"
    value {
      thresholds {
        slicing_specs {
        }
        threshold {
          value_threshold {
            lower_bound {
              value: 0.6
            }
          }
        }
      }
    }
  }
}
, data_location='<user provided PCollection>', file_format='<unknown>', model_location='gs://ihr-vertex-pipelines/payoneer/pipeline/237148598933/fraud-detect-pipeline-20220227130439/Trainer_-2901316516584620032/model/Format-Serving')

In [34]:
tfma.view.render_slicing_metrics(eval_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

In [32]:
tfma.view.render_plot(eval_result)

TypeError: argument of type 'NoneType' is not iterable

In [11]:
blessing = [x for x in artifacts if x.display_name == 'blessing'][0]

In [12]:
blessing

name: "projects/237148598933/locations/europe-west4/metadataStores/default/artifacts/7493935592862427244"
display_name: "blessing"
uri: "gs://ihr-vertex-pipelines/payoneer/pipeline/237148598933/fraud-detect-pipeline-20220227143821/Evaluator_6702046738829541376/blessing"
etag: "1645969239304"
create_time {
  seconds: 1645969106
  nanos: 195000000
}
update_time {
  seconds: 1645969239
  nanos: 304000000
}
state: LIVE
schema_title: "tfx.ModelBlessing"
schema_version: "0.0.1"
metadata {
  fields {
    key: "blessed"
    value {
      number_value: 1.0
    }
  }
  fields {
    key: "current_model"
    value {
      string_value: "gs://ihr-vertex-pipelines/payoneer/pipeline/237148598933/fraud-detect-pipeline-20220227130439/Trainer_-2901316516584620032/model"
    }
  }
  fields {
    key: "current_model_id"
    value {
      string_value: "projects/237148598933/locations/europe-west4/metadataStores/default/artifacts/2117142252301022551"
    }
  }
  fields {
    key: "tfx_version"
    value {
